# Análisis de Ofertas Relámpago en Mercado Libre

## ¿Qué es una Oferta Relámpago?

Las **Ofertas Relámpago** de Mercado Libre son promociones especiales de duración limitada —generalmente algunas horas— en las que se ofrecen productos seleccionados con descuentos importantes y un stock limitado. Estas campañas buscan generar un pico de ventas en poco tiempo, aumentar la visibilidad de ciertos productos e incentivar la compra impulsiva por parte de los usuarios.

Durante el tiempo que están activas, se muestra el porcentaje de unidades ya vendidas, generando un sentido de urgencia que impulsa la conversión. Esta estrategia se basa en la escasez y la presión temporal como mecanismos de marketing.

## Objetivo del Análisis

El objetivo de este ejercicio es realizar un Análisis Exploratorio de Datos (EDA) sobre el archivo `ofertas_relampago.csv`, el cual contiene información histórica sobre Ofertas Relámpago en un país determinado y durante un período específico. A través de este análisis se buscarán **insights relevantes sobre el comportamiento de estas promociones**, como patrones de éxito, estacionalidad, categorías más afectadas, entre otros aspectos.

> Este análisis forma parte del proceso de selección para el equipo de Data & Analytics de Mercado Libre

In [3]:
import pandas as pd

In [4]:
df_offer = pd.read_csv('../../../data/00_raw/ofertas_relampago.csv')

Vamos empezar analisando infos mas generales como la structura de lo dato, número de nulos, shape y otros.

In [5]:
df_offer.sample(3)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
11588,2021-07-27,2021-07-27 07:00:00+00:00,2021-07-27 13:00:00+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PHARMACEUTICS,BEAUTY & HEALTH,MLM-ESSENTIAL_OILS
17960,2021-07-10,2021-07-10 19:00:00+00:00,2021-07-10 19:00:04+00:00,lightning_deal,10,10,NaN,NaN,NaN,free_shipping,APPAREL ACCESORIES,APP & SPORTS,MLM-HANDBAGS
41304,2021-06-25,2021-06-25 13:00:00+00:00,2021-06-25 19:00:03+00:00,lightning_deal,15,15,NaN,NaN,NaN,free_shipping,PERSONAL CARE,BEAUTY & HEALTH,MLM-MAKEUP_BRUSHES


In [6]:
df_offer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48746 entries, 0 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OFFER_START_DATE           48746 non-null  object 
 1   OFFER_START_DTTM           48746 non-null  object 
 2   OFFER_FINISH_DTTM          48746 non-null  object 
 3   OFFER_TYPE                 48746 non-null  object 
 4   INVOLVED_STOCK             48746 non-null  int64  
 5   REMAINING_STOCK_AFTER_END  48746 non-null  int64  
 6   SOLD_AMOUNT                24579 non-null  float64
 7   SOLD_QUANTITY              24579 non-null  float64
 8   ORIGIN                     11316 non-null  object 
 9   SHIPPING_PAYMENT_TYPE      48746 non-null  object 
 10  DOM_DOMAIN_AGG1            48746 non-null  object 
 11  VERTICAL                   48746 non-null  object 
 12  DOMAIN_ID                  48746 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usag

- Tenemos columnas relacionadas a tiempo y que están como objects, tenemos columnas de stock y lo stock restante.
- Tenemos columnas de sales tanto en cantidad como en $, estas contienen nulos y vamos analisar lo motivo de contener nulos, pueden ser que no tuvieran ventas.
- Tenmos datos de origin, como fue la entrega y <b> datos de agrupamento. </b>
- Para agrupamiento es importante hablar que Vertical -> Dominio 1 -> Dominio ID, o sea vertical contiene tantos dominio 1 y dominio 1 tantos dominio ID
- Vamos tener que transformar los tipo de datos de tiempo para datetime

In [11]:
df_offer.describe(include='all')

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
count,48746,48746,48746,48746,48746.000000,48746.000000,24579.000000,24579.000000,11316,48746,48746,48746,48746
unique,61,783,7578,1,NaN,NaN,NaN,NaN,1,2,36,9,1266
top,2021-07-30,2021-07-30 13:00:00+00:00,2021-07-06 13:00:01+00:00,lightning_deal,NaN,NaN,NaN,NaN,A,free_shipping,HOME&DECOR,APP & SPORTS,MLM-SURGICAL_AND_INDUSTRIAL_MASKS
freq,1300,414,77,48746,NaN,NaN,NaN,NaN,11316,26658,6742,13241,1759
mean,NaN,NaN,NaN,NaN,35.007508,30.565216,51.208898,10.851052,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,206.761058,195.813806,175.254414,45.475305,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,-192.000000,0.280000,1.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,5.000000,4.000000,5.730000,1.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,10.000000,8.000000,12.420000,3.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,15.000000,15.000000,30.925000,6.000000,NaN,NaN,NaN,NaN,NaN


## Resumen del Dataset

- Total de registros: **48.746**
- Tipo de oferta: siempre **"lightning_deal"**
- Fechas: la más común es el **30/07/2021** (1.300 ofertas)
- Stock involucrado promedio: **35 unidades**
- Stock restante promedio: **30 unidades** (con valores negativos que podrían ser errores)
- Cantidad vendida promedio: **11 unidades**
- Monto vendido promedio: **$51**
- Tipo de envío: principalmente **"free_shipping"**
- Categoría más común: **HOME&DECOR**
- Vertical más común: **APP & SPORTS**
- Dominio más frecuente: **MLM-SURGICAL_AND_INDUSTRIAL_MASKS**
- Los tipos de ofertas son solamente lightning deal entonces voy a deletarlo en lo notebook de prep
- Añadir un tipo 'Otros' para la columna de ORIGIN